# Q
1. List comprehension 같은 것도 지원함?
2. 지원하는 모든 명령어를 사용한 복잡한 예제 제공 요청
3. +=, *= 등도 지원 필요?
4. 다른 방식으로 작성한 C++은? 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import compyler as cp
from utils import *
#from compyler.cipher_node import CipherNode

In [23]:
cpp_code = load_cpp_fun("./source_cpp/example1.cpp")

print(cpp_code)

compile_cpp_fun(cpp_code)

Ciphertext fun(Ciphertext a) {
    // Create FHE circuit
    auto b = -a + 2;
    auto c = a - b;

    return c;
Full Python code:

import compyler as cp
program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40)
    # Create FHE circuit
b = -a + 2
c = a - b
program.cppcompile(c)

Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json


In [24]:
cpp_code = load_cpp_fun("./source_cpp/example2.cpp")

print(cpp_code)

compile_cpp_fun(cpp_code)

Ciphertext fun(Ciphertext a) {
    // Create FHE circuit
    auto b = -a + 2;
    auto c = a;

    for (int i = 0; i < 5; ++i) {
        c = c * (-c + 2);
        b = b * (-c + 2);
    }

    auto d = b.mean() + c.mean();
    return d;
Full Python code:

import compyler as cp
program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40)
    # Create FHE circuit
b = -a + 2
c = a
for i in range(0, 5, 1):
    c = c * (-c + 2)
    b = b * (-c + 2)
d = b.mean() + c.mean()
program.cppcompile(d)

Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json


In [ ]:
cpp_code = load_cpp_fun("./source_cpp/example1.cpp")

### Python version

In [25]:
cpp_code = """
Ciphertext fun(Ciphertext a) {
    auto b = -a + 2;
    auto c = a - b;

    return c;
}
"""


compile_cpp_fun(cpp_code)

Full Python code:

import compyler as cp
program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40)
b = -a + 2
c = a - b
program.cppcompile(c)

Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json


In [26]:
cpp_code = """
Ciphertext fun(Ciphertext a) {
    // Create FHE circuit
    auto b = -a + 2;
    auto c = b.var();
    auto d = a.mean() - c;

    return d;
}
"""

compile_cpp_fun(cpp_code)

Full Python code:

import compyler as cp
program = cp.FheProgram(scheme="CKKS", devices=[1])
a = program.add_secret("a", 40)
    # Create FHE circuit
b = -a + 2
c = b.var()
d = a.mean() - c
program.cppcompile(d)

Wrote JSON to file /home/hoseung/.cache/fhecompiler/test.json
